# Vergleich von Imputation Methoden

An dieser Stelle sollen verschiede Methoden zum interpolieren von fehlenden Werten betrachtet und verglichen werden.

### Vorbereitung

In [65]:
import pandas as pd
import numpy as np
import math

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error


In der Folge werden keine Regionen oder Gruppierungen von Ländern und nur die Jahre ab 1990 betrachtet. 
Von dem verbliebenen Datensatz werden nur jene Indikatoren behalten, die mindesten 20% gefüllt sind.

In [42]:
def reset_base():
    pd.set_option('display.float_format', lambda x: '%.4f' % x)

    base= pd.read_csv('../Data/WDIData.csv') #see downloads worldbank
    base = base.drop(['Country Code', 'Indicator Code', 'Unnamed: 66'], axis=1) #name of column 'Unnamed: 66' may differ

    countries = pd.read_csv('additional_data/countries.csv').drop('Unnamed: 0', axis=1)
    base = pd.merge(base, countries, how='left')
    base = base.loc[base['Type'] != 'Region'].drop('Type', axis=1)

    base = base.set_index(['Country Name', 'Indicator Name'])
    base = base.loc[:, '1990':'2020']

    idx = pd.IndexSlice
    keep = pd.DataFrame(pd.DataFrame(base.isna().groupby('Indicator Name').sum()).T.sum(), columns=['NaN'])
    keep = keep.loc[keep['NaN'] <len(base.index.get_level_values('Country Name').unique())*len(base.columns)*0.8] #kept if 80% of entries are not NaN
    base = base.loc[idx[:, keep.index], :]
    return base

In [43]:
base = reset_base()
base.isna().sum().sum()

2758915

Um die Performanz unterschiedlicher Imutation Verfahren zu vergleichen werden weitere 1000 vorhandene (nicht NaN) Einträge entfernt. Diese 1000 Einträge werden später als Test-Daten verwendet, auf ihrer Grundlage lassen sich die Fehler der analysierten Verfahren errechnen. Um sicherzustellen, dass die Ergebnisse reproduzierbar sind wird ein Random State gesetzt. Dann werden zufällig Koordinaten zu Dateneinträgen gezogen. Da an dieser Stelle nur vorhandene Einträge relevant sind, werde zunächst zu viele Koordinaten gezogen, diese dann mit dem Datensatz abgeglichen und gelöscht, falls sie zu einem NaN zeigen und dann die ersten 1000 verbliebenen (und damit relevanten) Einträge ausgewählt. Diese werden in den Trainingsdaten entfernt. Auf diese Weise bleibt eine Reproduzierbarkeit erhalten.

### Simulation fehlender Werte und Evaluation

In [44]:
#random state to ensure reproducibility
rnds = np.random.RandomState(999)

#coordinates for data entries to be removed randomly
#5000 entries are selected
cords = pd.DataFrame([[rnds.randint(0, len(base), size=5000)[i], 
              rnds.randint(0, len(base.columns), size=5000)[i]]
              for i in range(5000)])

#all coordinates pointing to NaN entries are removed and
#first 1000 remaining entries are selected
cords['value'] = [base.iloc[cords[0][i], cords[1][i]] for i in cords.index]
cords = cords.dropna()[:1000].reset_index(drop=True)

In [45]:
#getting train data by changing randomly chosen values to NaN
def reset_train():
    train = base.copy()
    for i in cords.index:
        train.iloc[cords[0][i], cords[1][i]] = None
    return train

In [46]:
def evaluate(df):
    #scaling original data and imputed data
    #necessary ?????????????????????????????????????
    res1 = (pd.DataFrame({'y_true': [base.iloc[cords[0][i], cords[1][i]] for i in cords.index],
                        'y_pred': [df.iloc[cords[0][i], cords[1][i]] for i in cords.index],
                        'indicator': [df.index.get_level_values('Indicator Name')[cords[0][i]] for i in cords.index],
                        'year': [df.columns[cords[1][i]] for i in cords.index]})
         )
    
    
    scaler = StandardScaler().fit(train) #fitting on train?
    norm_base = pd.DataFrame(scaler.transform(base))
    df = pd.DataFrame(scaler.transform(df))

    #getting imputed values for simulated NaNs and true value 
    res =pd.DataFrame({'y_true': [norm_base.iloc[cords[0][i], cords[1][i]] for i in cords.index],
                       'y_pred': [df.iloc[cords[0][i], cords[1][i]] for i in cords.index]
                      })
    res = res.dropna()

   
    #calculate evaluation metrics
    r2 = r2_score(res['y_true'], res['y_pred'])
    rmse = math.sqrt(mean_squared_error(res['y_true'], res['y_pred']))
    still_missing = df.isna().sum().sum()
    
    print(f'Mit dieser Methode bleiben {still_missing} NaNs bestehen.')
    print('')
    print(f'{len(res)} Werte wurden für die Metriken verwendet.')
    print(f'r2: {r2}, rmse: {rmse}')

    return still_missing, r2, rmse
    #return res1

###  Imputation Verfahren

Es werden diese Imputation Methoden verglichen:
- Backcasting
- Durchschnitt
- regionaler Durchschnitt

#### Backfill

In [47]:
def impute_backfill(df):
    df = df.fillna(method='bfill', limit=3)
    return df

In [48]:
base = reset_base()
train = reset_train()
df3= impute_backfill(train) 

In [49]:
evaluate(df3)

Mit dieser Methode bleiben 962168 NaNs bestehen.

956 Werte wurden für die Metriken verwendet.
r2: -0.017399825459009977, rmse: 0.3421192666313887


(962168, -0.017399825459009977, 0.3421192666313887)

#### Durchschnitt des Indikators über alle Jahre hinweg

In [50]:
def impute_overall_means(df):
    #fill NaNs with overall mean of that indicator
    values = pd.DataFrame(df.stack()).groupby('Indicator Name')[0].mean()
    df = pd.DataFrame(df.stack(dropna=False))
    
    df[0] = df[0].fillna(df.groupby('Indicator Name')[0].transform('mean'))
    df = df.unstack()
    df.columns = df.columns.droplevel(0)
    df = df5.sort_index(level='Indicator Name')
        
    return df

In [51]:
base = reset_base()
train = reset_train()
df1 = impute_overall_means(train)

In [52]:
evaluate(df1)

Mit dieser Methode bleiben 0 NaNs bestehen.

1000 Werte wurden für die Metriken verwendet.
r2: -0.27079907766456746, rmse: 0.3738648822249713


(0, -0.27079907766456746, 0.3738648822249713)

#### Durchschnitt des Indikators für das jeweilige Jahr

In [53]:
def impute_yearly_means(df):
    #fill NaNs with overall mean of that indicator
    
    for i in df.columns:
        df[i] = df[i].fillna(df.groupby('Indicator Name')[i].transform('mean'))
            
    return df

In [54]:
base = reset_base()
train = reset_train()
df2 = impute_yearly_means(train)

In [55]:
evaluate(df2)

Mit dieser Methode bleiben 549010 NaNs bestehen.

1000 Werte wurden für die Metriken verwendet.
r2: -0.1390895839539641, rmse: 0.4250060604245091


(549010, -0.1390895839539641, 0.4250060604245091)

#### Regionaler Durchschnitt des Indikators für das jeweilige Jahr

#### MICE

In [58]:
percentage = []
continuous =[]
for i in base.index.get_level_values('Indicator Name').unique():
    if '%' in i:
        percentage.append(i)
    else:
        continuous.append(i)
        


In [64]:
idx = pd.IndexSlice

percentage = base.copy().loc[idx[:, percentage], :]
continuous = base.copy().loc[idx[:, continuous], :]


,,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Country Name,Indicator Name,,,,,,,,,,,,,,,,,,,,,
Afghanistan,Access to clean fuels and technologies for cooking (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.5000,23.0000,24.8000,26.7000,28.6000,30.3000,32.2000,34.1000,36.0000,NaN
Albania,Access to clean fuels and technologies for cooking (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69.0000,71.2000,73.3000,74.7000,75.9000,77.6000,78.8000,79.7000,80.7000,NaN
Algeria,Access to clean fuels and technologies for cooking (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,99.2000,99.3000,99.3000,99.3000,99.4000,99.3000,99.3000,99.3000,99.3000,NaN
American Samoa,Access to clean fuels and technologies for cooking (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,Access to clean fuels and technologies for cooking (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Virgin Islands (U.S.),Women's share of population ages 15+ living with HIV (%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
West Bank and Gaza,Women's share of population ages 15+ living with HIV (%),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Yemen, Rep.",Women's share of population ages 15+ living with HIV (%),42.9000,42.3000,41.7000,41.1000,40.6000,40.3000,40.3000,40.2000,40.2000,40.2000,...,38.3000,38.2000,38.1000,38.0000,37.8000,37.7000,37.6000,37.5000,37.3000,37.2000
